In [1]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow as tf

In [13]:
print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Users\\Admin\\AppData\\Roaming\\Python\\Python38\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [4]:
num_classes=5
img_rows,img_cols=48,48
batch_size=2




In [5]:
train_dat_dir="train"
validation_dat_dir="validation"

train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4
                                ,height_shift_range=0.4,horizontal_flip=True,vertical_flip=True,fill_mode="nearest")
validation_datagen=ImageDataGenerator(rescale=1./255)



In [6]:
train_generator=train_datagen.flow_from_directory(
                             train_dat_dir,color_mode="grayscale",target_size=(img_rows,img_cols),batch_size=batch_size,
                                class_mode='categorical',shuffle=True)

validation_generator=train_datagen.flow_from_directory(
                              validation_dat_dir,color_mode="grayscale",target_size=(img_rows,img_cols),batch_size=batch_size
                                ,class_mode='categorical',shuffle=True) 

Found 242 images belonging to 5 classes.
Found 242 images belonging to 5 classes.


In [45]:
train_generator.class_indices

{'Angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}

In [7]:
model=Sequential()

In [8]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [9]:
pip install --upgrade Pillow --user


Requirement already up-to-date: Pillow in c:\users\admin\anaconda3\lib\site-packages (8.1.0)
Note: you may need to restart the kernel to use updated packages.


In [46]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_little_vgg_color.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 26
nb_validation_samples = 8
epochs=5000

history=model.fit(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/5000
13/13 [==============================] - 6s 129ms/step - loss: 3.7479 - accuracy: 0.0288 - val_loss: 2.0968 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.09685, saving model to Emotion_little_vgg_color.h5
Epoch 2/5000
13/13 [==============================] - 1s 86ms/step - loss: 2.9619 - accuracy: 0.0911 - val_loss: 2.3868 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 2.09685
Epoch 3/5000
13/13 [==============================] - 1s 78ms/step - loss: 2.8766 - accuracy: 0.2346 - val_loss: 5.2118 - val_accuracy: 0.1250

Epoch 00003: val_loss did not improve from 2.09685
Epoch 4/5000
13/13 [==============================] - 1s 97ms/step - loss: 2.6850 - accuracy: 0.0348 - val_loss: 3.5496 - val_accuracy: 0.2500
Restoring model weights from the end of the best epoch.

Epoch 00004: val_loss did not improve from 2.09685

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 00004: early stopping
